In [1]:
from flask import Flask, render_template, request, redirect, url_for, flash
import psycopg2
import os
import datetime

## Settings

In [2]:
app =  Flask(__name__)
app.secret_key = 'mysecretkey'

app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['MAX_CONTENT_LENGTH'] = 16*1024*1024

## Conexion a Base de Datos

In [3]:
db_host = 'colocar tu host'
db_database = 'colocar tu base de datos'
db_user = 'colocar tu usuario'
db_password = 'colocar tu contraseña'

def connect_postgresql():
    
    conn = psycopg2.connect(
        host=db_host,
        database=db_database,
        user=db_user,
        password=db_password
    )
    
    return conn

## Utilities

In [4]:
def es_archivo_permitido(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ['png','jpg','jpeg','gif']

In [5]:
def convertir_a_binario(filename):
    with open(filename, 'rb') as f:
        blob = f.read()
        
    blob = psycopg2.Binary(blob)
    
    return blob

# Consultas a Base de Datos

In [6]:
def select_autolecturas():
    query = """
    SELECT a.idautolectura, b.dni, c.cuenta,
    a.fechacargafoto, a.horacargafoto, a.fotocargada,
    a.fechaobtencionlectura, a.horaobtencionlectura, a.lecturaobtencion, a.estadoactivo,
    a.idpersona, a.idsuministro
    FROM autolectura a, persona b, suministro c
    WHERE a.idsuministro = c.idsuministro
    AND a.idpersona = b.idpersona
    """

    conn = connect_postgresql()
    cur = conn.cursor()
    cur.execute(query) 
    
    list_persona = cur.fetchall()

    cur.close()
    conn.close()
    
    return list_persona

## Flask

In [7]:
@app.route('/')

def index():
    
    data = select_autolecturas()

    return render_template('index.html', autolecturas = data)

In [8]:
@app.route('/add_autolectura', methods = ['POST'])

def add_autolectura():
    
    try:
        
        if request.method == 'POST':
            
           
            if 'file' not in request.files:
                flash('No hay archivo seleccionado')
                return redirect(url_for('index'))
            
            file = request.files['file']
            
            if file.filename == '':
                flash('No hay imagen seleccionada')
                return redirect(url_for('index'))
            
            if file and es_archivo_permitido(file.filename):
                
                data1 = file.filename
                
                fullfilename = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
                
                file.save(fullfilename)
                
                blob = convertir_a_binario(fullfilename)
                
                idpersona = request.form['idpersona']
                idsuministro = request.form['idsuministro']
            
                now = datetime.datetime.now()
            
                fechacargafoto = str(now.year) + str('00' + str(now.month))[-2:] + str('00' + str(now.day))[-2:]
                horacargafoto = str('00' + str(now.hour))[-2:] + str('00' + str(now.minute))[-2:] + str('00' + str(now.second))[-2:]
            
                query1 = """UPDATE autolectura
                SET estadoactivo = 0
                WHERE idpersona = {} AND idsuministro = {} AND estadoactivo = 1
                """.format(idpersona,
                           idsuministro
                          )
                
                query2 = """INSERT INTO autolectura (
                idautolectura,
                idpersona,
                idsuministro,
                fechacargafoto,
                horacargafoto,
                fotocargada,
                estadoactivo) 
                VALUES (DEFAULT, {}, {}, '{}', '{}', {}, 1)""".format(idpersona,
                                                                      idsuministro,
                                                                      fechacargafoto,
                                                                      horacargafoto,
                                                                      blob
                                                                     )
                
                #query de actualizacion
                conn = connect_postgresql()
                cur = conn.cursor()
        
                cur.execute(query1)
                
                conn.commit()
                
                cur.close()
                conn.close()
                
                #query de insercion
                conn = connect_postgresql()
                cur = conn.cursor()
                
                cur.execute(query2)
                
                conn.commit()
        
                count = cur.rowcount
        
                cur.close()
                conn.close()

                flash(str(count) + ' imagen cargada correctamente')
                
                data2 = select_autolecturas()
                
                return render_template('index.html', filename = data1, autolecturas = data2)
            else:
                
                flash('Archivos permitidos .png .jpg .jpeg .gif')
                
                return redirect(url_for('index'))

    except Exception as e:
        
        print(e)
        
    return redirect(url_for('index'))

In [9]:
@app.route('/display_image/<string:filename>')

def display_image(filename):
    return redirect(url_for('static', filename = 'uploads/' + filename), code = 301)

## Ejecucion

In [10]:
#Ejecuta el servidor en el puerto 5000
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jul/2021 02:25:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2021 02:25:45] "POST /add_autolectura HTTP/1.1" 200 -
